## uncomment this and run in the terminal if you need a venv

<!-- create the venv -->
<!-- Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope Process -->
<!-- py -m venv .venv -->
<!-- .venv\scripts\activate -->
<!-- py -m pip install --upgrade pip -->
<!-- py -m pip install Faker -->
<!-- py -m pip install datetime -->
<!-- py -m pip install pandas -->
<!-- py -m pip install numpy -->
<!-- py -m pip install duckdb==0.7.0 -->

<!-- pip list -->

## Import some packages

In [227]:
import pandas as pd
# pd.set_option('display.max_rows', 100)
import random
import names
import numpy as np
import datetime
from faker import Faker

# fake date source: https://stackoverflow.com/questions/553303/generate-a-random-date-between-two-other-dates

# uncomment and use your favorite seed if you want it to be reproducible
# random.seed(94025)
# Faker.seed(94025)
fake = Faker()

## Make Q1 tables

In [228]:
team_count = 50
team_names = []

for i in range(0,team_count):
    team_names.append("Team " + fake.last_name())

# bonus points if the interviewee asks whether they're distinct
# if you want another layer of complexity, comment out the next line for a random chance of duplicate teams
team_names = np.unique(team_names)

q1_table_teams = pd.DataFrame({"id":list(range(1,len(team_names)+1)),
                     "team":team_names})


In [229]:
match_count = 100
match_participants = []

for i in range(0,match_count):
    match_participants.append(random.sample(sorted(q1_table_teams['id']), 2))

q1_table_matches = pd.DataFrame({"match_id":list(range(1,len(match_participants)+1)),
                       "winner":[z[0] for z in match_participants],
                       "loser":[z[1] for z in match_participants]})

# this could use some data quality checks, e.g. 
#   make sure everyone plays at least a few games


## Make Q2 tables

In [230]:
customer_count = 100
number_of_purchases = 10000

customer_names = []
purchase_dates = []
purchasing_customers = []
purchased_items = []

# start by faking some names
for i in range(0,customer_count):
    customer_names.append(fake.name())
customer_names = np.unique(customer_names)

# now let's fake some purchases, then assign a random customer name and item
for i in range(0,number_of_purchases):
    purchase_dates.append(fake.date_between(start_date='-180d', end_date='today').isoformat())
    purchasing_customers.append(random.sample(sorted(customer_names), 1)[0])
    purchased_items.append(random.sample(['Widget 1','Widget 2','Widget 3'], 1)[0])

q2_table_purchases = pd.DataFrame({"date":purchase_dates,
                                   "customer":purchasing_customers,
                                   "item":purchased_items})

In [231]:
non_customer_count = 100
total_number_of_touches = 100000 # trying to keep this accessible so someone could replicate in excel if needed
channels = ['Direct','Email','Organic','Referral','Unknown'] # ???

touched_customer_names = []
non_customer_names = []
touch_dates = []
touch_channels = []

# just to keep it fun, there should be some marketing touches that didn't convert/end up in the purchases table
for i in range(0,non_customer_count):
    non_customer_names.append(fake.name())

# we already removed duplicates from `customers` but
#   there could be duplicates in non_customers
#   there could also be overlap between the two sets
# so just to be safe:
all_customers = np.unique(list(set(non_customer_names).union(set(customer_names))))


for i in range(0,total_number_of_touches):
    touch_dates.append(fake.date_between(start_date='-180d', end_date='today').isoformat())
    touched_customer_names.append(random.sample(sorted(all_customers), 1)[0])
    touch_channels.append(random.sample(channels, 1)[0])

# there's a chance we have no marketing entry before the purchase date
# not worth fixing
# if someone checks/notices/cares about that, they've probably already passed

q3_table_marketing = pd.DataFrame({"date":touch_dates,
                                   "customer":touched_customer_names,
                                   "channel":touch_channels})


In [234]:
q1_table_teams.to_csv('tables/teams.csv', index=False)
q1_table_matches.to_csv('tables/matches.csv', index=False)
q2_table_purchases.to_csv('tables/purchases.csv', index=False)
q3_table_marketing.to_csv('tables/marketing.csv', index=False)